### dependencies

In [1]:
import numpy as np
from numpy import concatenate
from pandas import read_csv, DataFrame, concat
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn
import sklearn
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import statsmodels
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [ ]:
# Define global variables
#TODO find out what is the target variable and what lag steps
num_features = 53
target_variable = 'POILBRE'
lag_steps = 1


In [11]:

#TODO check if data is clean already
# df = read_csv('data/dataset.csv', header = 0, index_col = 0, usecols = [i for i in range(0, num_features+1)])
df = read_csv('../../data/djia_stock_data.csv')
df.dropna(inplace=True) # NA values after June 2017
print(df.shape)
df.head()


(2516, 181)


,Price,Adj Close,Adj Close.1,Adj Close.2,Adj Close.3,Adj Close.4,Adj Close.5,Adj Close.6,Adj Close.7,Adj Close.8,...,Volume.20,Volume.21,Volume.22,Volume.23,Volume.24,Volume.25,Volume.26,Volume.27,Volume.28,Volume.29
0,Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,...,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT
2,2015-01-02,24.3204345703125,120.22659301757812,15.425999641418457,80.13386535644531,113.6572036743164,70.11015319824219,58.91096115112305,20.18903923034668,73.4642562866211,...,27913900,4985800,113680000,7251400,1700400,1270800,3060900,8389600,11421200,13505400
3,2015-01-05,23.635284423828125,118.79789733886719,15.10949993133545,78.01464080810547,112.87007141113281,66.40930938720703,57.846900939941406,19.786867141723633,70.52776336669922,...,39673900,6889200,197952000,8626100,2161800,1728700,4679000,12751200,18964500,20937000
4,2015-01-06,23.637510299682617,114.97055053710938,14.76449966430664,76.35202026367188,111.5406265258789,65.98199462890625,56.882286071777344,19.779552459716797,70.4951171875,...,36447900,7576000,197764000,7791200,2133000,2350900,3468300,11070000,22950100,24615300
5,2015-01-07,23.96895980834961,118.98588562011719,14.920999526977539,78.01956176757812,113.27236938476562,67.00447845458984,56.61378860473633,19.962358474731445,70.43641662597656,...,29114100,7256000,321808000,5986600,2487300,1558200,3225800,9346800,20793600,25495200


In [ ]:
for i in df.columns:
    print(i)
# 10 companies. columns include Close, 

Price
Close
Close.1
Close.2
Close.3
Close.4
Close.5
Close.6
Close.7
Close.8
Close.9
Close.10
Close.11
Close.12
Close.13
Close.14
Close.15
Close.16
Close.17
Close.18
Close.19
Close.20
Close.21
Close.22
Close.23
Close.24
Close.25
Close.26
Close.27
Close.28
Close.29
High
High.1
High.2
High.3
High.4
High.5
High.6
High.7
High.8
High.9
High.10
High.11
High.12
High.13
High.14
High.15
High.16
High.17
High.18
High.19
High.20
High.21
High.22
High.23
High.24
High.25
High.26
High.27
High.28
High.29
Low
Low.1
Low.2
Low.3
Low.4
Low.5
Low.6
Low.7
Low.8
Low.9
Low.10
Low.11
Low.12
Low.13
Low.14
Low.15
Low.16
Low.17
Low.18
Low.19
Low.20
Low.21
Low.22
Low.23
Low.24
Low.25
Low.26
Low.27
Low.28
Low.29
Open
Open.1
Open.2
Open.3
Open.4
Open.5
Open.6
Open.7
Open.8
Open.9
Open.10
Open.11
Open.12
Open.13
Open.14
Open.15
Open.16
Open.17
Open.18
Open.19
Open.20
Open.21
Open.22
Open.23
Open.24
Open.25
Open.26
Open.27
Open.28
Open.29
Volume
Volume.1
Volume.2
Volume.3
Volume.4
Volume.5
Volume.6
Volume.7
Volume.8
Vol

Transformation for LSTM

In [ ]:
# Transform the time-series dataset specifically for LSTM
def time_series_to_lstm(df, target_variable, lag_steps=1, dropna=True, fill='ffill'):
    """
    Transforms time-series data into a supervised learning format compatible with LSTMs.
    
    Args:
        df (pd.DataFrame): The input time-series dataset.
        target_variable (str): The column to predict.
        lag_steps (int): Number of past time steps to include.
        dropna (bool): Whether to drop rows with NaN values.
        fill (str): What to fill NaN values with ('ffill', 'bfill', 'mean', 'median', None)
        
    Returns:
        df_transformed (pd.DataFrame): DataFrame to be used as input for LSTM model
    """
    # Ensure dataframe format
    if isinstance(df, list):
        df = DataFrame(df)
    
    cols = []
    feature_names = []
    
    # Create Lag Steps
    for i in range(lag_steps, 0, -1):
        cols.append(df.shift(i))
        feature_names += [f"{col}(t-{i})" for col in df.columns]
    
    # Current time step (t) for target variable
    cols.append(df[[target_variable]])
    feature_names += [f"{target_variable}(t)"]
    
    # Combine and assign column names
    df_transformed = concat(cols, axis=1)
    df_transformed.columns = feature_names

    # Drop NaN rows if required
    if dropna:
        df_transformed.dropna(inplace=True)
    else:
        if not None:
            df_transformed.fillna(method=fill, inplace=True)
    
    return df_transformed

In [ ]:
df_transformed = time_series_to_lstm(df, target_variable, lag_steps)
print(df_transformed.shape)
df_transformed.head()
# for company in companies: 
    # df_transformed.to_csv(f'data/{company}_transformed.csv')
# Export transformed dataset
df_transformed.to_csv('data/dataset_transformed.csv')


In [ ]:
# Train-Test Split
split = int(df_transformed.shape[0]*0.8)
train = df_transformed[:split]
test = df_transformed[split:]

print('Split Shape:', train.shape, test.shape)
# Scale to avoid distance calculation bias
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test) # Avoid data leakage
train.iloc[:, -1]


In [ ]:
# Separate into features and target (last column)
X_train, y_train = train_scaled[:, :-1], train_scaled[:, -1]
X_test, y_test = test_scaled[:, :-1], test_scaled[:, -1]

In [ ]:
# Reshape features for LSTM
X_train = X_train.reshape((X_train.shape[0], lag_steps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], lag_steps, X_test.shape[1]))

print('Train Shape:', X_train.shape, y_train.shape)
print('Test Shape:', X_test.shape, y_test.shape)

## LSTM training

In [ ]:
# Initiate LSTM model
model = Sequential()
model.add(LSTM(125, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
# Model fitting
history = model.fit(X_train, y_train, epochs=70, batch_size=128, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Plot training progression
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

## Predictions

In [ ]:
# Get predicted values
yhat = model.predict(X_test)

# Reshaping back into 2D for inverse scaling
X_test_inv = X_test.reshape((X_test.shape[0], X_test.shape[2])) 

# Concatenate and Inverse Scaling
# Prediction
inv_yhat = concatenate((X_test_inv, yhat), axis=1) # Required to get back original scale
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, num_features*lag_steps] # Extract target_variable

# Validation
y_test_inv = y_test.reshape((len(y_test), 1))
inv_y = concatenate((X_test_inv, y_test_inv), axis=1) # Both arrays must have same dimensions
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, num_features*lag_steps]

In [ ]:
# Get MAE
mae = np.sqrt(mean_absolute_error(inv_y, inv_yhat))
print('LSTM Test MAE: %.3f' % mae)

In [ ]:
plt.plot(inv_y, label = 'Actual')
plt.plot(inv_yhat, label = 'Predicted')
plt.legend()
plt.show()